Copyright (C) 2022  Kamil Erguler <k.erguler@cyi.ac.cy>
 
This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details (<https://www.gnu.org/licenses/>). 

In [1]:
using sPop2
using Plots
using Statistics
using Distributions

In [3]:
N = 10.0
mu = 20.0
sd = 5.0

mu2 = 0.0

xr = 0:50

function simDet(mode)
    a = newPop(false, mode)
    addPop(a, 0, 0.0, N)
    retd = [0 a.size]
    for n in [1:50;]
        if mu2 > 0.0 && n >= mu2
            tmp = stepPop(a, mu2, sd, 0.0)
        else
            tmp = stepPop(a, mu, sd, 0.0)
        end
        retd = vcat(retd, [n tmp["size"]])
    end
    return retd
end

function simStoch(mode)
    rets = []
    for r in [1:1000;]
        a = newPop(true, mode)
        addPop(a, 0, 0.0, N)
        ret = [a.size]
        for n in [1:50;]
            if mu2 > 0.0 && n >= mu2
                tmp = stepPop(a, mu2, sd, 0.0)
            else
                tmp = stepPop(a, mu, sd, 0.0)
            end
            push!(ret, tmp["size"])
        end
        if rets == []
            rets = ret
        else
            rets = cat(rets,ret,dims=2)
        end
    end
    res = []
    for r in 1:size(rets)[1]
        p = quantile(rets[r,:],[0.05,0.5,0.95])
        if res == []
            res = p
        else
            res = hcat(res, p)
        end
    end
    return hcat(0:50,transpose(res))
end

for key in ["AGE_GAMMA", "AGE_NBINOM", "AGE_FIXED", "ACC_FIXED", "ACC_ERLANG", "ACC_PASCAL"]
    det = simDet(key)
    if !(key === "AGE_FIXED" || key === "ACC_FIXED")
        stoch = simStoch(key)
    end

    if key === "ACC_ERLANG" || key === "AGE_GAMMA"
        theta = sd * sd / mu
        k = mu / theta
    elseif key === "AGE_NBINOM"
        theta = mu / (sd * sd)
        k = mu * theta / (1.0 - theta)
    elseif key === "AGE_FIXED"
        theta = 1.0
        k = mu
    elseif key === "ACC_PASCAL"
        theta = mu / (sd * sd)
        k = mu * theta / (1.0 - theta)
    end

    if !(key === "AGE_FIXED" || key === "ACC_FIXED")
        plot(stoch[:,1], stoch[:,4], fillrange = stoch[:,2], c="#9ecae1", label="Stoch. (90% range)")
        plot!(stoch[:,1], stoch[:,3], lw=4, c="#3182bd", label="Stoch. (median)")
        plot!(det[:,1], det[:,2], line = :scatter, c="black", ms=8, label="Deterministic")
    else
        plot(det[:,1], det[:,2], line = :scatter, c="black", ms=8, label="Deterministic")
    end
    if key === "ACC_ERLANG" || key === "AGE_GAMMA"
        plot!(xr, N*(1.0 .- cdf(Gamma(k,theta),xr)), line= :scatter, c="gray", ms=6, label="Expected")
    elseif key === "ACC_PASCAL" || key === "AGE_NBINOM"
        plot!(xr, N*(1.0 .- cdf(NegativeBinomial(k,theta),xr .- 1)), line= :scatter, c="gray", ms=6, label="Expected")
    end
    savefig(string("figures/distribution_",key,".pdf"))
end

In [5]:
mu = [40.0 20.0]
sd = [5.0 5.0]
theta = sd .* sd ./ mu
k = mu ./ theta

xri = collect(0:199)
xr = collect(0:0.01:50)

a = newPop(false, "ACC_ERLANG")
addPop(a, 0, 0.0, 100.0)
ii = [0 1]
retA = [0 a.size]
retL = [0 readKeys(a.devc)]
for n in xri[2:end]
    i = n<=20 ? 1 : 2
    stepPop(a, mu[i], sd[i], 0.0)
    retA = vcat(retA, [n a.size])
    retL = vcat(retL, [n readKeys(a.devc)])
    ii = vcat(ii,[n i])
end

plot(xri,retA[:,2], c="black", lw=2, legend=false)
for i in [2 1]
    plot!(xr,100*(1.0 .- cdf(Gamma(k[i],theta[i]), xr)), c="gray", lw=3, ls=:dash, legend=false)
end
xlims!((0, 50))
savefig(string("figures/switch.pdf"))

plot(retL[1,2][2], c="black", legend = false)
for r in 2:50
    plot!(retL[r,2][2], c="black", lw=retL[r,1] in [5 20 29] ? 3 : 1, legend = false)
end
ylims!((0,20))
savefig("figures/switch_devc.pdf")